## Importing packages

In [31]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import os
from skimage.io import imread
from skimage.color import rgb2lab, lab2rgb
from PIL import Image

# Preparing the dataset
1.Importing images

2.Converting images into Grayscale

In [40]:
direc='C:/Users/2019/Desktop/Studying/Machine Learning/Image_recolorizer/tiny-imagenet-200/train/n01443537/images2'
counter=0
for x in os.listdir(direc):
    if x.endswith('.JPEG') and counter<10:
        counter+=1
        path=direc+'/'+x
        img=imread(path)
        
        plt.figure(figsize=(64,64))
        
        img=rgb2lab(img)
        img[...,1] = img[...,2] = 0
        img=lab2rgb(img)
        
#         plt.imshow(img)
        img = Image.fromarray(img, 'RGB')
        savepath=direc+'/'+x.replace('.JPEG', 'lab.JPEG')
        img.save(savepath,'JPEG')
        

<Figure size 4608x4608 with 0 Axes>

<Figure size 4608x4608 with 0 Axes>

<Figure size 4608x4608 with 0 Axes>

<Figure size 4608x4608 with 0 Axes>

<Figure size 4608x4608 with 0 Axes>

<Figure size 4608x4608 with 0 Axes>

<Figure size 4608x4608 with 0 Axes>

<Figure size 4608x4608 with 0 Axes>

<Figure size 4608x4608 with 0 Axes>

<Figure size 4608x4608 with 0 Axes>